- save and load as  classmethod?
- overload constructor?

In [1]:
with open('datasets/shakespeare.txt') as file:
    text = file.read()

In [2]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import FunctionTransformer
# from unidecode import unidecode
# from tokenizer.bpe_tokenizer import BytePairEncodingTokenizer

# tokenizer = BytePairEncodingTokenizer(0)
# tokenizer.load('tokenizer/trained_tokenizers/bpe.pkl')

# pipeline = Pipeline([
#     ('unicode_to_ascii', FunctionTransformer(unidecode)),
#     ('encode', FunctionTransformer(tokenizer.encode))
# ])

# pipeline.transform('lowest')

In [3]:
from tokenizer.bpe_tokenizer import BytePairEncodingTokenizer

text = text[:50000]

tokenizer = BytePairEncodingTokenizer(1000)
tokenizer.fit([text])

print(tokenizer.vocab_size)

1095


In [4]:
import torch

data = torch.tensor(tokenizer.encode(text), dtype=torch.long)

n = int(0.9 * len(data))

train_data = data[:n]
test_data = data[n:]


In [5]:
context_length = 64
batch_size = 64

def get_batch():
    idx = torch.randint(len(data) - context_length, (batch_size, ))
    x = torch.stack([data[i:i + context_length] for i in idx])
    y = torch.stack([data[i + 1:i + context_length + 1] for i in idx])
    return x, y

In [6]:
from language_model.model import Encoder

embed_size = 512
num_layers = 6
num_heads = 8
forward_expansion = 4
dropout = 0

model = Encoder(
    tokenizer.vocab_size,
    embed_size,
    context_length,
    num_layers,
    num_heads,
    forward_expansion,
    dropout
)

print(sum([p.numel() for p in model.parameters()]) / 1e6, 'M parameters')

20.061255 M parameters


In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.05)

for iter in range(50):
    if iter % 5 == 0:
        print(iter)

    x_batch, y_batch = get_batch()

    logits, loss = model(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1,1), dtype=torch.long)
print(tokenizer.decode(model.generate(context, max_new_tokens=500)[0].tolist()))

0
5
10
15
20
25
30
35
40
45
 arth
seCw show thee T

;esAu F
 keepthe atstononinthth pte'd,ionnot:,
VIRGILIA:hreow
c thedcome 
reodw:adoorlC
Il
Marciushus ,ghepthe  mo ueyeedaamfolethey le AUFIDIUS:heacavecabon wcome
elseesgo c er aLARTIUS: edearwre  brweattheve ,neandthate  nean batfiseu
tonAflya 
;theersfa
rtheMes
 a -ctiou gm noblesay atc   am
hath
 carry ahuhm ati , sayinof My
VOLUMNIA:WCome,he Firstuswellpeople,Ion.'comehMARCIUS:vatrs!con
 madam;is  
conitiMarciusIarsNthata Marcius ;
'tmy herf poorUSinut
we fe'Tishe inionWisyou
ow: LARTIUS:wis ding
matter isMaredreo' a atisirc leleinlebes
havepo awwhich
ouWld rnans  vnisandased, erisour hekbutfemathiw
  the on umbutr'tuegtheube do 
mionno ce
worthyto IbonkambulesTend ones
seels!sword to
aeny. go
liat.roer atyatHa
restalreadHowesupsuf  es orittimealove ofbe     welldasdo ithethlne ap
 have  itfahem,enourfiwinonomEvenld:thesayconrihanots. s  
enLARTIUS: Ofwillswethetledmight   th  .l s we tiyy   
